1. 以下关于集成学习的说法不正确的是：   
   A. 在集成学习中，我们可以为数据空间的不同区域使用不同的预测模型，再将预测结果进行组合。   
   B. 一组预测模型可以有多种方式进行集成学习。   
   C. 有效的集成学习需要集合中的模型具有单一性，最好将同一类型的预测模型结合起来。   
   D. 训练集成模型时，单个模型的参数不会随之更新。  



答：  
A. 正确。数据空间的不同部分可能有比较大差别的模式，在不同区域学习不同的预测模型再组合起来就可以预测这个数据空间的数据了。  
B. 正确。一组预测模型可以用不同的方法集成达到不同的效果，比如Bagging，Boosting，Stacking等。  
C. 错误。集成学习就是集合不同结构类型的模型才能获得更好的性能的。  
D. 正确。单个模型的参数是训练单个模型的时候确定好。集成学习模型训练是将训练好的模型结合起来。


---

   2. 以下关于提升算法的说法正确的是：   
A. AdaBoost算法中，$err$绝对值越小的模型权重绝对值越大，在集成模型中占有主导地位。   
B. AdaBoost算法中，需要按照之前学习器的结果对训练数据进行加权采样。   
C. GBDT算法用到了“梯度反方向是函数值下降最快方向”的思想。   
D. GBDT的正则化约束只考虑了叶节点的数目  

答：  
A. 正确。AdaBoost是通过错误率来确定每个学习器的权重的，错误率越小权重就越大，就在模型中起到比较重要的影响。  
B. 正确。AdaBoost会给训练数据加权进行调整，使得前一轮的错误样本在下一轮更加被关注。  
C. 正确。这是GBDT的核心思想。将梯度和决策树结合，每次迭代拟合决策树模型进行更新。  
D. 错误。GBDT的正则化约束还考虑了树的深度，叶结点的输出值等，综合约束来降低过拟合。


---

3. 由基础学习器提取特征后再供给元学习器进一步学习，这一特征提取的思想在前面哪些章节也出现过？为什么合适的特征提取往往能提升算法的表现？

答：特征提取可以从大量的特征洪提取合适的特征，减少特征的维度和噪声，找到更加能拟合数据模式的重要特征，达到更好的泛化效果。而且特征提取可以降低计算的复杂度，消除一些不必要的关系影响。  
特征提取在许多地方都出现过，比如CNN中的对图像数据的提取（池化），RNN中对序列中潜在关系的提取（GRU），SVM中的特征映射，决策树中的对数据的分割提取等等。

---

4. 基于本章代码，尝试非线性的元分类器，如神经网络和决策树，观察原始数据拼接到新数据上的模型预测性能的改变。

非线性元分类器能捕捉更复杂的数据关系，尤其是对于非线性特征。原始数据拼接新数据后，模型的性能有所下降，可能是因为原始特征中带有噪音和不相关的信息干扰了元学习。不过影响并不大，说明基分类器已经基本上学习到了特征信息。

In [17]:
# 导入需要的库
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.model_selection import train_test_split

In [18]:
# 创建随机数据集
X, y = make_classification(
    n_samples = 1000, # 样本数量
    n_features = 16, # 每个样本的特征数量
    n_informative = 5, # 与输出y有关联的特征量
    n_redundant = 2, # 对输出y没贡献的特征量
    n_classes = 2, # 两个类别
    flip_y = 0.1, # 标签的错误率
    random_state = 2024
)

print(X.shape)
print(y.shape)

(1000, 16)
(1000,)


In [19]:
from sklearn.model_selection import KFold
from sklearn.base import clone

# 堆垛分类器，继承sklearn中的集成分类器基类EnsembleClassifier
class StackingClassifier():

    def __init__(
        self, 
        classifiers, # 基础分类器
        meta_classifier, # 元分类器
        concat_feature=False, # 是否将原始数据拼接在新数据上
        kfold=5 # K折交叉验证
    ):
        self.classifiers = classifiers
        self.meta_classifier = meta_classifier
        self.concat_feature = concat_feature
        self.kf = KFold(n_splits=kfold)
        # 为了在测试时计算平均，我们需要保留每个分类器
        self.k_fold_classifiers = []
        
    def fit(self, X, y):
        # 用X和y训练基础分类器和元分类器
        n_samples, n_features = X.shape
        self.n_classes = np.unique(y).shape[0]
        
        if self.concat_feature:
            features = X
        else:
            features = np.zeros((n_samples, 0))
        for classifier in self.classifiers:
            self.k_fold_classifiers.append([])
            # 训练每个基础分类器
            predict_proba = np.zeros((n_samples, self.n_classes))
            for train_idx, test_idx in self.kf.split(X):
                # 交叉验证
                clf = clone(classifier)
                clf.fit(X[train_idx], y[train_idx])
                predict_proba[test_idx] = clf.predict_proba(X[test_idx])
                self.k_fold_classifiers[-1].append(clf)
            features = np.concatenate([features, predict_proba], axis=-1)
        # 训练元分类器
        self.meta_classifier.fit(features, y)
    
    def _get_features(self, X):
        # 计算输入X的特征
        if self.concat_feature:
            features = X
        else:
            features = np.zeros((X.shape[0], 0))
        for k_classifiers in self.k_fold_classifiers:
            k_feat = np.mean([clf.predict_proba(X)
                for clf in k_classifiers], axis=0)
            features = np.concatenate([features, k_feat], axis=-1)
        return features
    
    def predict(self, X):
        return self.meta_classifier.predict(self._get_features(X))
        
    def score(self, X, y):
        return self.meta_classifier.score(self._get_features(X), y)

In [20]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.neural_network import MLPClassifier
from sklearn import tree

# 划分训练集和测试集
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=0)

# 基础分类器
rf = RFC(n_estimators=10, max_features='sqrt', 
    random_state=0).fit(X_train, y_train)
knc = KNC().fit(X_train, y_train)
# multi_class='ovr'表示二分类问题
lr = LR(solver='liblinear', multi_class='ovr', 
    random_state=0).fit(X_train, y_train)
print('随机森林：', rf.score(X_test, y_test))
print('KNN：', knc.score(X_test, y_test))
print('逻辑斯谛回归：', lr.score(X_test, y_test))


# 元分类器
meta_lr = LR(solver='liblinear', multi_class='ovr', random_state=0) # 线性元分类器
meta_DTC = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3) # 决策树元分类器
meta_nn = MLPClassifier(hidden_layer_sizes=(16,), random_state=1)

# -------------------- #
# 元分类器为线性回归
sc_lr = StackingClassifier([rf, knc, lr], meta_lr, concat_feature=False)
sc_lr.fit(X_train, y_train)
print('Stacking分类器（线性）：', sc_lr.score(X_test, y_test))

# 带原始特征的stacking分类器
sc_lr_concat = StackingClassifier([rf, knc, lr], meta_lr, concat_feature=True)
sc_lr_concat.fit(X_train, y_train)
print('带原始特征的Stacking分类器（线性）：', sc_lr_concat.score(X_test, y_test))

# -------------------- #
# 元分类器为决策树
sc_dtc = StackingClassifier([rf, knc, lr], meta_DTC, concat_feature=False)
sc_dtc.fit(X_train, y_train)
print('Stacking分类器（决策树）：', sc_dtc.score(X_test, y_test))

# 带原始特征的stacking分类器
sc_dtc_concat = StackingClassifier([rf, knc, lr], meta_DTC, concat_feature=True)
sc_dtc_concat.fit(X_train, y_train)
print('带原始特征的Stacking分类器（决策树）：', sc_dtc_concat.score(X_test, y_test))

# -------------------- #
# 元分类器为MLP
sc_nn = StackingClassifier([rf, knc, lr], meta_nn, concat_feature=False)
sc_nn.fit(X_train, y_train)
print('Stacking分类器（神经网络）：', sc_nn.score(X_test, y_test))


# 带原始特征的stacking分类器
sc_nn_concat = StackingClassifier([rf, knc, lr], meta_nn, concat_feature=True)
sc_nn_concat.fit(X_train, y_train)
print('带原始特征的Stacking分类器（神经网络）：', sc_nn_concat.score(X_test, y_test))


随机森林： 0.795
KNN： 0.83
逻辑斯谛回归： 0.74
Stacking分类器（线性）： 0.84
带原始特征的Stacking分类器（线性）： 0.825
Stacking分类器（决策树）： 0.825
带原始特征的Stacking分类器（决策树）： 0.815
Stacking分类器（神经网络）： 0.84
带原始特征的Stacking分类器（神经网络）： 0.83


d:\miniconda\envs\d2l\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


---

5. 在提升算法中，弱学习器的数量越多，元学习器的效果是否一定越好？调整AdaBoost和GBDT代码中弱学习器的数量，验证你的想法。

In [27]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier


# 初始化stump
stump = DTC(max_depth=1, min_samples_leaf=1, random_state=0)

# 弱分类器个数
M = np.arange(1, 101, 5)
bg_score = []
rf_score = []
dsc_ada_score = []
real_ada_score = []
plt.figure()

with tqdm(M) as pbar:
    for m in pbar:
        # bagging算法
        bc = BaggingClassifier(base_estimator=stump, 
            n_estimators=m, random_state=0)
        bc.fit(X_train, y_train)
        bg_score.append(bc.score(X_test, y_test))
        # 随机森林算法
        rfc = RandomForestClassifier(n_estimators=m, max_depth=1, 
            min_samples_leaf=1, random_state=0)
        rfc.fit(X_train, y_train)
        rf_score.append(rfc.score(X_test, y_test))
        # 离散 AdaBoost，SAMME是分步加性模型（stepwise additive model）的缩写
        dsc_adaboost = AdaBoostClassifier(base_estimator=stump, 
            n_estimators=m, algorithm='SAMME', random_state=0)
        dsc_adaboost.fit(X_train, y_train)
        dsc_ada_score.append(dsc_adaboost.score(X_test, y_test))
        # 实 AdaBoost，SAMME.R表示弱分类器输出实数
        real_adaboost = AdaBoostClassifier(base_estimator=stump, 
            n_estimators=m, algorithm='SAMME.R', random_state=0)
        real_adaboost.fit(X_train, y_train)
        real_ada_score.append(real_adaboost.score(X_test, y_test))

# 绘图
plt.plot(M, bg_score, color='blue', label='Bagging')
plt.plot(M, rf_score, color='red', label='Random Forest')
plt.plot(M, dsc_ada_score, color='green', label='Discrete AdaBoost')
plt.plot(M, real_ada_score, color='purple', label='Real AdaBoost')
plt.xlabel('Number of trees')
plt.ylabel('Test score')
plt.legend()
plt.show()

  0%|          | 0/20 [00:00<?, ?it/s]


TypeError: __init__() got an unexpected keyword argument 'base_estimator'

<Figure size 640x480 with 0 Axes>

---

6. 基于xgboost库，对于本章涉及的回归任务，调试树的数量上限、每棵树的深度上限、学习率，观察其训练模型性能的改变，讨论是大量较浅的树组成的GBDT模型更强，还是少量的较深的树组成的GBDT模型更强。